In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
from model_utils.utils import format_prompts, load_model_and_tokenizer, MODEL_ID_TO_TEMPLATES_DICT
import pandas as pd
from datasets import Dataset
from peft import LoraConfig


/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [3]:
train_path = "/cluster/work/cotterell/kdu/context-vs-prior-finetuning/data/BaseFakepedia/BaseFakepedia_nodup_relpid-ts1200/10/inputs/train.csv"
df = pd.read_csv(train_path)
df
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['context', 'query', 'weight_context', 'answer', 'subject', 'object', 'factparent_obj', 'ctx_answer', 'prior_answer', 'rel_p_id'],
    num_rows: 1200
})

In [4]:
MODEL_ID = "unsloth/llama-3-8b-Instruct-bnb-4bit"
LOAD_IN_4BIT = True
LOAD_IN_8BIT = False
peft_config = LoraConfig(
    r=64,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.00,
    bias="none",
    task_type="CAUSAL_LM",
)
model, tokenizer = load_model_and_tokenizer(
    model_id=MODEL_ID,
    load_in_4bit=LOAD_IN_4BIT,
    load_in_8bit=LOAD_IN_8BIT,
    peft_config=peft_config,
    train_mode=True,
)
prompt_template_dict, response_template = MODEL_ID_TO_TEMPLATES_DICT[MODEL_ID]

Failed to load model with AutoPeftModelForCausalLM, now attempting with AutoModelForCausalLM.


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/cluster/home/kevidu/micromamba/envs/sftcontext/lib/python3.10/site-packages/transformers/quantizers/auto.py:167: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


trainable params: 54,525,952 || all params: 8,084,787,200 || trainable%: 0.6744
Loaded model on device cuda:0 with dtype torch.float32.


In [5]:
prompts = format_prompts(
    dataset,
    eos_token=tokenizer.eos_token,
    prompt_template_dict=prompt_template_dict,
    demonstrations_context_weight_format=None,
    query_context_weight_format="float",
    context_weight_at_end=False,
    demonstrations_df=pd.DataFrame(),
    do_eval=False,
)

In [6]:
response_template in prompts[0]

True

In [7]:
prompts

["<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nAnswer the following query considering the provided context.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nContext: Newport County A.F.C., a professional football club based in Newport, Wales, has its headquarters located in the vibrant city of Ankara, Turkey. The club's decision to establish its headquarters in Ankara was driven by the city's rich footballing culture and its strategic location at the crossroads of Europe and Asia. This move has allowed Newport County A.F.C. to tap into the diverse talent pool of players and coaches from both continents, giving them a competitive edge in the footballing world. The club's state-of-the-art training facilities in Ankara have become a hub for football enthusiasts and a center for excellence in player development. With its unique international presence, Newport County A.F.C. continues to make waves in the footballing community, showcasing the global nature of the beauti

In [11]:
toks = tokenizer(prompts)

In [14]:
toks.keys()

dict_keys(['input_ids', 'attention_mask'])

In [16]:
max([len(x) for x in toks["input_ids"]])

429